In [ ]:
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup

In [39]:
CHECK_INTERVAL_SEC = 60 * 5
CHECK_DOW = ['土','日']
ticket_sales_url = 'https://www.tokyodisneyresort.jp/ticket/sales_status/'
table_id = 'calendarTableSp ticketStock'

In [ ]:
year_monthes = [datetime.date.today().strftime('%Y%m'),(datetime.date.today() + pd.DateOffset(days=31)).strftime('%Y%m')]
year_monthes

In [ ]:
browser = webdriver.Chrome(executable_path = 'D:\\Python\\lib\\chromedriver.exe')

In [29]:
table_df_old = dict()

In [36]:
# ページ情報取得
year_month = year_monthes[1]
browser.get(ticket_sales_url+year_month)
html = BeautifulSoup(browser.page_source, 'html.parser')
table = html.findAll(class_ = table_id)[0]

table_df = None
for row in table.findAll(['tr']):
    
    if len(row.findAll('th')) > 0:
        columns = []
        for th in row.findAll('th'):
            columns.append(th['class'][0])

        table_df = pd.DataFrame(columns=columns)

    elif len(row.findAll('td')) > 0:
        day = row.findAll('td')[0].find('div').get_text()      
        tdl_status = ' '.join(row.findAll('td')[1]['class'])
        tds_status = ' '.join(row.findAll('td')[2]['class'])
        data = [day,tdl_status,tds_status]

        table_df = table_df.append(dict(zip(table_df.columns,data)), ignore_index=True)

    
table_df = (
    table_df
    .pipe(lambda df: 
          df[['tdl','tds']]
          .join(
              df.day.apply(lambda x: pd.Series(data=[pd.to_datetime(year_month+x.split('(')[0].zfill(2)),x.split('(')[1][0]],index=['date','dow'] ))
          )
         )
    .set_index('date')
)

In [34]:
# table_df_old に値が格納されている場合
if year_month in table_df_old.keys():
    



not None


In [64]:
f"""{(
    table_df_old[year_month].loc[lambda df: df.dow.isin(CHECK_DOW)]
    .join(table_df[['tdl','tds']],rsuffix='_new')
    .loc[lambda df: (df.tdl != df.tdl_new) & (df.tdl_new.isin(['tdl','tdl is-few']))]
    .replace({'tdl':'〇','tdl is-few':'△'})
    .loc[:,['dow','tdl_new']]
)}"""

'           dow tdl_new\ndate                  \n2021-06-12   土       〇'

In [59]:
(
    table_df_old[year_month].loc[lambda df: df.dow.isin(CHECK_DOW)]
    .join(table_df[['tdl','tds']],rsuffix='_new')
    .loc[lambda df: (df.tds != df.tds_new) & (df.tds_new.isin(['tds','tds is-few']))]
    .replace({'tds':'〇','tds is-few':'△'})
    .loc[:,['dow','tds_new']]
)

,dow,tds_new
date,,
2021-06-12,土,△
2021-06-20,日,〇


In [47]:
table_df.loc['2021-06-12','tdl'] = 'tdl'
table_df.loc['2021-06-12','tds'] = 'tds is-few'
table_df.loc['2021-06-20','tds'] = 'tds'

In [37]:
table_df_old[year_month] = table_df.copy()